In [1]:
%matplotlib notebook
from matplotlib import pyplot as plt
import pandas
import re
import nltk
import numpy as np
from numpy import *
import csv
import theano.tensor as T
import os.path
from nltk.collocations import *
from optparse import OptionParser
from collections import Counter
from copy import copy
import cPickle
import csv
import warnings

from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score

from keras.layers.convolutional import MaxPooling1D, Convolution1D
from keras.layers.recurrent import LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Graph
from keras.engine.training import slice_X
from keras.layers.core import Layer, Dense, Dropout, Activation,\
    Reshape, Flatten, Lambda
from keras.regularizers import Regularizer
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.optimizers import Adadelta
from keras.callbacks import Callback


from IPython.utils.io import CapturedIO
from gensim.models import Word2Vec
from pkg_resources import resource_filename
import utils
import datasets
from unidecode import unidecode

# Yoon Kim's tokenization
def my_process(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^\w(),|!?\'\`\:\-\.;\$%#]", " ", string)
    string = re.sub(r"\'s", " is", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"(?<=\w)\.\.\.", " ... ", string)
    string = re.sub(r"(?<=\w)\.", " . ", string)
    string = re.sub(r"(?<=\w),", " , ", string)
    string = re.sub(r"(?<=\w);", " ; ", string)
    string = re.sub(r"(?<=\w)!", " ! ", string)
    string = re.sub(r"\((?=\w)", " ( ", string)
    string = re.sub(r"(?<=\w)\)", " ) ", string)
    string = re.sub(r"(?<=\w)\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip()

def mixed_score(y_true, y_probs):
    thresholds = sorted(unique(y_probs))
    max_f1, best_threshold = 0, 0
    for th in thresholds:
        f1 = f1_score(y_true, y_probs >= th)*1)
        if f1 > max_f1:
            max_f1 = f1
            best_th = th
    th = best_th    
    return {
        "precision": y_true[y_probs >= th].sum()*1./(y_probs >= th).sum(),
        "recall": y_true[y_probs >= th].sum()*1./y_true.sum(),
        "f1": f1_score(y_true, (y_probs >= th)*1),
        "auc": roc_auc_score(y_true, y_probs)
    }

# This function chooses the best threshold based on f1 of validation.
def seq_score(model, X, y):
    val_split = model.last_fit_params.get('validation_split', 0.)
    split_at = int(model.last_fit_X.shape[0] * (1. - val_split))
    X_val, y_val = model.last_fit_X[split_at:], model.last_fit_y[split_at:]
    val_probs = model.predict(X_val).flatten()
    thresholds = sorted(unique(val_probs))
    max_f1, best_threshold = 0, 0
    for threshold in thresholds:
        f1 = f1_score(y_val, (val_probs >= threshold)*1)
        if f1 > max_f1:
            max_f1 = f1
            best_threshold = threshold
    return mixed_score(y, model.predict(X).flatten(), best_threshold)
    
# Same as seq_f1 but for graph model
def graph_score(model, data):
    val_split = model.last_fit_params.get('validation_split', 0.)
    split_at = int(model.last_fit_data['output'].shape[0] * (1. - val_split))
    data_val = {k: slice_X(v, split_at) for k, v in model.last_fit_data.items()}
    val_probs = model.predict(data_val)['output'].flatten()
    thresholds = sorted(unique(val_probs))
    max_f1, best_threshold = 0, 0
    for threshold in thresholds:
        f1 = f1_score(data_val['output'], (val_probs >= threshold)*1)
        if f1 > max_f1:
            max_f1 = f1
            best_threshold = threshold
    return mixed_score(data['output'], 
                       model.predict(data)['output'].flatten(), 
                       best_threshold)

def seq_auc(model, X, y):
    preds = model.predict(X).flatten()
    return roc_auc_score(y, preds)
    
def graph_auc(model, data):
    preds = model.predict(data)['output'].flatten()
    return roc_auc_score(data['output'], preds)

seq_eval_f = seq_score
graph_eval_f = graph_score
results = pandas.DataFrame()

SyntaxError: invalid syntax (<ipython-input-1-f43faf411e5b>, line 73)

In [40]:
def mixed_score(y_true, y_probs, th):  
    y_true, y_probs = asarray(y_true), asarray(y_probs)
    return {
        "precision": y_true[y_probs >= th].sum()*1./(y_probs >= th).sum(),
        "recall": y_true[y_probs >= th].sum()*1./y_true.sum(),
        "f1": f1_score(y_true, (y_probs >= th)*1),
        "auc": roc_auc_score(y_true, y_probs)
    }

# Load ADR Twitter data set

In [3]:
data_path = "/home/trung/data/psb2016/"
tweets, clean_tweets, labels = [], [], []
with open(os.path.join(data_path, 'tweets.txt')) as f:
    for line in f:
        user_id, tweet_id, label, tweet = line.strip().split('\t')
        tweets.append(unidecode(tweet.decode('utf-8')))
        labels.append(int(label))
    
np.random.seed(0)
# Shuffle the data as Keras won't shuffle validation data.
# This can make the training ends early as we are using
# early stop for regularisation.
idx = np.random.permutation(len(tweets))
tweets, labels = asarray(tweets)[idx], asarray(labels)[idx]
skf = list(StratifiedKFold(labels, n_folds=10))
texts = asarray(tweets, dtype='str')

In [45]:
labels.shape

(5108,)

In [44]:
labels.sum()

557

# The baselines

In [4]:
import pandas
from sklearn.linear_model import LogisticRegression
from zhang_adr.concept_matching import run_cm
from zhang_adr.maxent_tfidf import run_tfidf
from zhang_adr.maxent_nblcr import run_nblcr
from zhang_adr.maxent_we import run_we
from zhang_adr.tweetnlp import tweet_tagger
from zhang_adr.preprocess import clean_tweet

tokens, tags = tweet_tagger.runtagger_parse(texts)
zhang_clean_texts = []
for token, tag in zip(tokens, tags):
    zhang_clean_texts.append(clean_tweet(token, tag))
zhang_clean_texts = asarray(zhang_clean_texts)

In [41]:
import warnings
warnings.filterwarnings("ignore")

results = results[results['model'] != 'CM']
results = results[results['model'] != 'ME-TFIDF']
results = results[results['model'] != 'ME-NBLCR']
results = results[results['model'] != 'ME-WE']
clf = LogisticRegression(class_weight="auto")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i, (train_idx, test_idx) in enumerate(skf, 1):
        print "### Fold {}:".format(i)
        train, test = [], []
        for train_id in train_idx:
            train.append({"id": None, "label": labels[train_id], "text": zhang_clean_texts[train_id]})
        train = pandas.DataFrame(train)
        for test_id in test_idx:
            test.append({"id": None, "label": labels[test_id], "text": zhang_clean_texts[test_id]})
        test = pandas.DataFrame(test)

        result = {}

        y_pred_cm = run_cm(train, test, resource_filename('zhang_adr', 'data/ADR-lexicon.txt'))
        result = mixed_score(test['label'].values, y_pred_cm, 0.5)
        result['model'] = 'CM'
        results = pandas.concat([results, pandas.DataFrame([result])])

        _, y_prob_tfidf = run_tfidf(train, test, grams='123', n_dim=40000, clf=clf)
        result = mixed_score(test['label'].values, asarray(y_prob_tfidf[:, 1]), 0.5)
        result['model'] = 'ME-TFIDF'
        results = pandas.concat([results, pandas.DataFrame([result])])

        _, y_prob_nblcr = run_nblcr(train, test, 'nblcr', grams='123', clf=clf)
        result = mixed_score(test['label'].values, y_prob_nblcr[:, 1], 0.5)
        result['model'] = 'ME-NBLCR'
        results = pandas.concat([results, pandas.DataFrame([result])])

        _, y_prob_we = run_we(train, test, resource_filename('zhang_adr', 'data/w2v_150.txt'), 150, clf=clf)
        result = mixed_score(test['label'].values,  y_prob_we[:, 1], 0.5)
        result['model'] = 'ME-WE'
        results = pandas.concat([results, pandas.DataFrame([result])])

### Fold 1:
### Fold 2:


### Fold 3:


### Fold 4:


### Fold 5:


### Fold 6:


### Fold 7:


### Fold 8:


### Fold 9:


### Fold 10:

In [42]:
results.groupby('model').mean()

,auc,f1,precision,recall
model,,,,
CM,0.587609,0.230405,0.132295,0.892208
CNN,0.884428,0.507228,0.473284,0.568831
CNNA,0.868821,0.494731,0.403737,0.658636
CRNN,0.869807,0.507763,0.489378,0.552922
ME-NBLCR,0.832942,0.233311,0.791053,0.139805
ME-TFIDF,0.854756,0.449863,0.331677,0.700260
ME-WE,0.823112,0.398744,0.274590,0.728734
RCNN,0.867046,0.488006,0.433383,0.586851


# Our methods

In [6]:
# w2v = Word2Vec.load_word2vec_format(
#     '/home/trung/data/embeddings/word2vec_twitter_model/word2vec_twitter_model.bin', 
#     binary=True
# )

# w2v = Word2Vec.load_word2vec_format(
#     '/home/trung/data/embeddings/GoogleNews.bin', 
#     binary=True
# )

# w2v = Word2Vec.load_word2vec_format("/home/trung/data/embeddings/wlin/struc_skip_600.txt")

# w2v = Word2Vec.load_word2vec_format(
#     resource_filename('zhang_adr', 'data/w2v_150.txt'),
#     binary=False,
# )

w2v = Word2Vec.load_word2vec_format(
    '/home/trung/data/embeddings/glovec/tmp',
    binary=False
)

dim = w2v.layer1_size

In [7]:
from zhang_adr.TextUtility import TextUtility

MOST_FREQUENT_WORDS = 20000
USE_CACHE = False
INCLUDE_UNKNOWN_WORDS = False

docs = [[w for w in TextUtility.text_to_wordlist(text)\
         if INCLUDE_UNKNOWN_WORDS or w in w2v.index2word]\
         for text in zhang_clean_texts]
all_words = Counter([w for doc in docs for w in doc])
top_words = sorted(all_words.items(), key=lambda t: t[1], reverse=True)
top_words = top_words[:MOST_FREQUENT_WORDS]
V = {w:i for i, (w, freq) in enumerate(top_words)}
X = utils.vectorize(docs, V)

# initialize embedding matrix
my_embeddings = np.random.normal(-.25, .25, size=(X.max() + 1, dim))
for w in V:
    if w in w2v:
        my_embeddings[V[w]] = w2v[w]
        
# set embedding of padded character as 0s.
my_embeddings[len(V) + 1] = np.zeros((dim, ))

### Logistic regression of sum of embeddings

In [237]:
def T_sum(x, **kwargs):
    import theano.tensor as T
    return T.sum(x, axis=-2)
    
def mk_lr_model_f(max_len, embeddings, embedding_fixed=False, 
                  optimizer='adadelta', loss='binary_crossentropy'):
    
    def lr_model():
        m = Sequential()
        m.add((utils.FixedEmbedding if embedding_fixed else Embedding)\
                (*embeddings.shape, input_length=max_len, weights=[embeddings]))
        m.add(Lambda(T_sum, output_shape=(embeddings.shape[1],)))
        m.add(Dense(1, activation='sigmoid'))
        m.compile(loss=loss, optimizer=optimizer, class_mode='binary')
        return m
    
    return lr_model

In [ ]:
# sequential model
early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=1)
scores = utils.seq_cross_validate(
    mk_lr_model_f(X.shape[1], my_embeddings, optimizer='adadelta'),
    X, labels, 
    skf, eval_f=seq_eval_f,
    fit_params={
        "callbacks": [early_stopper],
        "validation_split": .1,
        "batch_size": 50,
        "nb_epoch": 100,
    })
results["my-lr-sum-embedding-dynamic-embedding"] = scores

### CNN

In [ ]:
# # sequential model
# early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=1)
# model = utils.mk_yk_model_f(X.shape[1], my_embeddings, embedding_fixed=True, n_filters=300)()
# train_idx, test_idx = skf5[0]
# fit_params={
#     "callbacks": [early_stopper],
#     "validation_split": .1,
#     "batch_size": 50
# }
# model.fit(X[train_idx], labels[train_idx], **fit_params)

# idx = test_idx[(model.predict(X[test_idx]) >= 0.5).flatten()*1 != labels[test_idx]]
# wrongs = pandas.DataFrame(data={"y": labels[idx], "docs": asarray(docs)[idx], 
#                                 "zhang_clean_texts": zhang_clean_texts[idx],
#                                 "texts": texts[idx]})
# wrongs.to_csv('incorrect.csv')

In [ ]:
# sequential model
early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=0)
scores = utils.seq_cross_validate(
    utils.mk_yk_model_f(X.shape[1], my_embeddings, n_filters=300),
    X, labels, 
    skf, eval_f=seq_eval_f,
    fit_params={
        "callbacks": [early_stopper],
        "validation_split": .1,
        "batch_size": 50
    }, 
    verbose=0)
df = pandas.DataFrame(scores)
model_name = "CNN"
df["model"] = model_name
results = pandas.concat([results[results["model"] != model_name], df])
df.mean()

Train on 4136 samples, validate on 460 samples
Epoch 1/10
4136/4136 [==============================] - 3s - loss: 0.3907 - val_loss: 0.3671
Epoch 2/10
4136/4136 [==============================] - 3s - loss: 0.3206 - val_loss: 0.3437
Epoch 3/10
4136/4136 [==============================] - 3s - loss: 0.2903 - val_loss: 0.3237
Epoch 4/10
4136/4136 [==============================] - 3s - loss: 0.2607 - val_loss: 0.3025
Epoch 5/10
4136/4136 [==============================] - 3s - loss: 0.2341 - val_loss: 0.2957
Epoch 6/10
4136/4136 [==============================] - 3s - loss: 0.2098 - val_loss: 0.2793
Epoch 7/10
4136/4136 [==============================] - 3s - loss: 0.1884 - val_loss: 0.2642
Epoch 8/10
4136/4136 [==============================] - 3s - loss: 0.1742 - val_loss: 0.2693
Epoch 9/10
4136/4136 [==============================] - 3s - loss: 0.1531 - val_loss: 0.2570
Epoch 10/10
4136/4136 [==============================] - 3s - loss: 0.1390 - val_loss: 0.2499
Train on 4137 samples,

auc          0.884428
f1           0.507228
precision    0.473284
recall       0.568831
dtype: float64

### GRU

In [62]:
reload(utils)

<module 'utils' from 'utils.py'>

In [ ]:
# sequential model
early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=0)
scores = utils.seq_cross_validate(
    utils.mk_gru_model_f(X.shape[1], my_embeddings),
    X[:, ::-1], labels,
    skf, eval_f=seq_eval_f,
    verbose=0,
    fit_params={
        "callbacks": [early_stopper],
        "validation_split": .1,
        "batch_size": 50
    })
df = pandas.DataFrame(scores)
model_name = "GRU"
df["model"] = model_name
results = pandas.concat([results[results["model"] != model_name], df])
df.mean()

Train on 4136 samples, validate on 460 samples
Epoch 1/10
 950/4136 [=====>........................] - ETA: 9s - loss: 0.4740

In [66]:
results.groupby('model').mean()

,auc,f1,precision,recall
model,,,,
CM,0.587609,0.230405,0.132295,0.892208
CNN,0.884428,0.507228,0.473284,0.568831
CNNA,0.868821,0.494731,0.403737,0.658636
CRNN,0.869807,0.507763,0.489378,0.552922
GRU,0.867040,0.491331,0.492879,0.513571
ME-NBLCR,0.832942,0.233311,0.791053,0.139805
ME-TFIDF,0.854756,0.449863,0.331677,0.700260
ME-WE,0.823112,0.398744,0.274590,0.728734
RCNN,0.867046,0.488006,0.433383,0.586851


### CRNN

In [ ]:
# sequential model
early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=0)
scores = utils.seq_cross_validate(
    utils.mk_cgru_model_f(X.shape[1], my_embeddings, nb_filter=300, rnn_output=300),
    X[:, ::-1], labels,
    skf, eval_f=seq_eval_f,
    verbose=0,
    fit_params={
        "callbacks": [early_stopper],
        "validation_split": .1,
        "batch_size": 50
    })
df = pandas.DataFrame(scores)
model_name = "CRNN"
df["model"] = model_name
results = pandas.concat([results[results["model"] != model_name], df])
df.mean()

Train on 4136 samples, validate on 460 samples
Epoch 1/10
4136/4136 [==============================] - 8s - loss: 0.3502 - val_loss: 0.3575
Epoch 2/10
4136/4136 [==============================] - 8s - loss: 0.2910 - val_loss: 0.3349
Epoch 3/10
4136/4136 [==============================] - 8s - loss: 0.2588 - val_loss: 0.3876
Epoch 4/10
4136/4136 [==============================] - 8s - loss: 0.2293 - val_loss: 0.2921
Epoch 5/10
4136/4136 [==============================] - 8s - loss: 0.2104 - val_loss: 0.2950
Epoch 6/10
4136/4136 [==============================] - 8s - loss: 0.1882 - val_loss: 0.2787
Epoch 7/10
4136/4136 [==============================] - 8s - loss: 0.1578 - val_loss: 0.2682
Epoch 8/10
4136/4136 [==============================] - 8s - loss: 0.1409 - val_loss: 0.2840
Epoch 9/10
4136/4136 [==============================] - 8s - loss: 0.1238 - val_loss: 0.2873
Epoch 10/10
4136/4136 [==============================] - 8s - loss: 0.1041 - val_loss: 0.2865
Train on 4137 samples,

### RCNN

In [ ]:
early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=0)
scores = utils.seq_cross_validate(
    utils.mk_rcnn_model_f(X.shape[1], my_embeddings, rnn_output=300, nb_filter=300, filter_length=5),
    X, labels,
    skf, eval_f=seq_eval_f,
    verbose=0,
    fit_params={
        "callbacks": [early_stopper],
        "validation_split": .1,
        "batch_size": 50
    })
df = pandas.DataFrame(scores)
model_name = "RCNN"
df["model"] = model_name
results = pandas.concat([results[results["model"] != model_name], df])
df.mean()

In [13]:
results.groupby("model").mean()

,auc,f1,precision,recall
model,,,,
CM,0.587609,0.230405,0.132295,0.892208
CNN,0.884428,0.507228,0.473284,0.568831
CNNA,0.868821,0.494731,0.403737,0.658636
CRNN,0.869807,0.507763,0.489378,0.552922
ME-NBLCR,0.832942,0.233311,0.791053,0.139805
ME-TFIDF,0.854756,0.449863,0.331677,0.700260
ME-WE,0.823112,0.398744,0.274590,0.728734
RCNN,0.867046,0.488006,0.433383,0.586851


### CNN with Attention

In [ ]:
early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=0)
scores = utils.graph_cross_validate(
    utils.mk_attention_based_model_f(X.shape[1], my_embeddings, attention_l2=0.1),
    {"tokens": X, "output": labels},
    skf,
    eval_f=graph_eval_f,
    verbose=0,
    fit_params={
        "callbacks": [early_stopper],
        "validation_split": .1,
        "batch_size": 50
    })
df = pandas.DataFrame(scores)
model_name = "CNNA"
df["model"] = model_name
results = pandas.concat([results[results["model"] != model_name], df])
df.mean()

In [140]:
m = utils.mk_attention_based_model_f(X.shape[1], my_embeddings)()
early_stopper = utils.MyEarlyStopping(monitor='val_loss', patience=5, verbose=1)
m.fit({"tokens": X[skf[0][0]], "output": labels[skf[0][0]]},
      callbacks=[early_stopper],
      batch_size=50,
      validation_split=.1)

Train on 4136 samples, validate on 460 samples
Epoch 1/100
4136/4136 [==============================] - 14s - loss: 0.3568 - val_loss: 0.3665
Epoch 2/100
4136/4136 [==============================] - 14s - loss: 0.3271 - val_loss: 0.3518
Epoch 3/100
4136/4136 [==============================] - 14s - loss: 0.3080 - val_loss: 0.3645
Epoch 4/100
4136/4136 [==============================] - 14s - loss: 0.2774 - val_loss: 0.3253
Epoch 5/100
4136/4136 [==============================] - 14s - loss: 0.2396 - val_loss: 0.2738
Epoch 6/100
4136/4136 [==============================] - 14s - loss: 0.2210 - val_loss: 0.2649
Epoch 7/100
4136/4136 [==============================] - 14s - loss: 0.1835 - val_loss: 0.2548
Epoch 8/100
4136/4136 [==============================] - 14s - loss: 0.1612 - val_loss: 0.3401
Epoch 9/100
4136/4136 [==============================] - 14s - loss: 0.1328 - val_loss: 0.2801
Epoch 10/100
4136/4136 [==============================] - 14s - loss: 0.1188 - val_loss: 0.2900
Ep

In [141]:
from keras import backend as K
attentions = K.function([m.nodes["embedding"].get_input()],
                         [m.nodes["norm_attn_weights"].get_output()])

features = K.function([m.nodes["embedding"].get_input()],
                         [m.nodes["features"].get_output()])

In [142]:
"had" in w2v.index2word

True

In [145]:
for i in xrange(1000):
    if len(docs[skf[0][0][i]]) >= 8 and labels[skf[0][0][i]]==1:
        print tweets[skf[0][0][i]]
        print zhang_clean_texts[skf[0][0][i]]        
        print zip(docs[skf[0][0][i]], attentions([X[skf[0][0][i:i+1]]])[0].flatten())
        print

@tittuck i know it made me want to be sick every time i had it plus fluoxetine is the bigges bunch of shit ive been on worse than sertraline
i know it made me want to be sick every time i had it plus fluoxetine is the bigges bunch of shit ive been on worse than sertraline
[(u'i', 0.0056520808), (u'know', 0.013411545), (u'it', 0.028869804), (u'made', 0.036486648), (u'me', 0.042846639), (u'want', 0.047599487), (u'to', 0.071960993), (u'be', 0.095526487), (u'sick', 0.12615407), (u'every', 0.094749615), (u'time', 0.055059109), (u'i', 0.025319701), (u'had', 0.016237818), (u'it', 0.01196784), (u'plus', 0.010993169), (u'fluoxetine', 0.0097043943), (u'is', 0.0093425903), (u'the', 0.008333168), (u'bigges', 0.0076636323), (u'bunch', 0.0090183076), (u'of', 0.011944612), (u'shit', 0.019051364), (u'ive', 0.028213581), (u'been', 0.040556699), (u'on', 0.047379613), (u'worse', 0.049805731), (u'than', 0.036138404), (u'sertraline', 0.021764973)]

@Shellyfanelly I was on #azathioprine for about 8 years, i

In [110]:
asarray(docs[skf[0][0][1:2]])
attentions([X[skf[0][0][1:2]]])[0]

array([[[ 0.09381618],
        [ 0.10483222],
        [ 0.08684829],
        [ 0.05685537],
        [ 0.03641567],
        [ 0.02486874],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848],
        [ 0.02484848]]], dtype=float32)